In [49]:
import pandas as pd
import csv
import numpy as np

df = pd.read_csv('dataset.csv', low_memory=False)
df.head()

,index,director_name,duration,actor_2_name,genres,actor_1_name,movie_title,num_voted_users,actor_3_name,movie_imdb_link,num_user_for_reviews,language,country,title_year,imdb_score
0,0,James Cameron,178.0,Joel David Moore,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,Wes Studi,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,2009.0,7.9
1,1,Gore Verbinski,169.0,Orlando Bloom,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,Jack Davenport,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,2007.0,7.1
2,2,Sam Mendes,148.0,Rory Kinnear,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,Stephanie Sigman,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,2015.0,6.8
3,3,Christopher Nolan,164.0,Christian Bale,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,Joseph Gordon-Levitt,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,2012.0,8.5
4,4,Doug Walker,NaN,Rob Walker,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,8,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,NaN,NaN,7.1


# movies

In [50]:
movies_df = pd.DataFrame()
movies_df['movie_id'] = df['index'] + 1
movies_df['movie_title'] = df['movie_title']
movies_df['release_date'] = df['title_year']
movies_df['language'] = df['language']
movies_df['directed_by'] = df['director_name']
movies_df.to_csv('movies.csv', index=False)

# actors
Mapping actor\_id to actor\_name

In [51]:
actors_distinct = {}
counter = 1 #actor_id 
for actor in df['actor_1_name']:
    if actor not in actors_distinct:
        actors_distinct[actor] = counter 
        counter += 1 
for actor in df['actor_2_name']:
    if actor not in actors_distinct:
        actors_distinct[actor] = counter 
        counter += 1 
for actor in df['actor_3_name']:
    if actor not in actors_distinct:
        actors_distinct[actor] = counter 
        counter += 1 

actors_df = pd.DataFrame.from_dict(actors_distinct, orient='index') 

actors_df  = actors_df.reset_index()

actors_df.columns = ['actor_name', 'actor_id'] 
actors_df = actors_df[['actor_id', 'actor_name']] 

print(actors_df)
actors_df.to_csv('actors.csv', index=False) 

      actor_id       actor_name
0            1      CCH Pounder
1            2      Johnny Depp
2            3  Christoph Waltz
3            4        Tom Hardy
4            5      Doug Walker
...        ...              ...
6251      6252   Sara Stepnicka
6252      6253  Daniella Pineda
6253      6254    Sam Underwood
6254      6255   David Chandler
6255      6256         Jon Gunn

[6256 rows x 2 columns]


# genres

Maps distinct genres to genre_id

In [52]:
genres_distinct = {}
counter = 1
for i in df['genres']:
    d=i.replace(',','').split('|') 
    for j in d:
      if j not in genres_distinct:
        genres_distinct[j] = counter 
        counter += 1 
print(genres_distinct) 

ids = genres_distinct.values()
genres = genres_distinct.keys()

genres_df = pd.DataFrame({'genre_id': ids, 'genre': genres})

genres_df.to_csv('genres.csv', index=False)

{'Action': 1, 'Adventure': 2, 'Fantasy': 3, 'Sci-Fi': 4, 'Thriller': 5, 'Documentary': 6, 'Romance': 7, 'Animation': 8, 'Comedy': 9, 'Family': 10, 'Musical': 11, 'Mystery': 12, 'Western': 13, 'Drama': 14, 'History': 15, 'Sport': 16, 'Crime': 17, 'Horror': 18, 'War': 19, 'Biography': 20, 'Music': 21, 'Game-Show': 22, 'Reality-TV': 23, 'News': 24, 'Short': 25, 'Film-Noir': 26}


# cast

In [53]:
cast_map = {}   
for i, row in df.iterrows():
    movie_id = row['index'] + 1
    cast_map[movie_id] = [row['actor_1_name'], row['actor_2_name'], row['actor_3_name']] 
data = []
for movie_id, actors in cast_map.items():
    for actor in actors:
        data.append({'movie_id': movie_id, 'actor_id': actors_distinct[actor]}) 

cast_df = pd.DataFrame(data)
print(cast_df)
cast_df.to_csv('cast.csv', index=False) 

       movie_id  actor_id
0             1         1
1             1      1727
2             1       496
3             2         2
4             2       181
...         ...       ...
15124      5042      2121
15125      5042      2986
15126      5043      2098
15127      5043      4130
15128      5043      6256

[15129 rows x 2 columns]


# movie_genres

In [54]:
movie_genres = pd.melt(df, id_vars=['index'], value_vars=['genres'], var_name='genre', value_name='movie_genres')

for i in movie_genres['movie_genres']:
    i = i.replace(',','').split('|') 
print(movie_genres)

      index   genre                     movie_genres
0         0  genres  Action|Adventure|Fantasy|Sci-Fi
1         1  genres         Action|Adventure|Fantasy
2         2  genres        Action|Adventure|Thriller
3         3  genres                  Action|Thriller
4         4  genres                      Documentary
...     ...     ...                              ...
5038   5038  genres                     Comedy|Drama
5039   5039  genres     Crime|Drama|Mystery|Thriller
5040   5040  genres            Drama|Horror|Thriller
5041   5041  genres             Comedy|Drama|Romance
5042   5042  genres                      Documentary

[5043 rows x 3 columns]


In [55]:
movie_genre_mapping = {}
for index, row in movie_genres.iterrows():
    movie_id = row['index'] + 1
    genres = row['movie_genres'].replace(',','').split('|')
    
    for genre in genres:
        genre_id = genres_distinct[genre]
        if movie_id not in movie_genre_mapping:
            movie_genre_mapping[movie_id] = []
        movie_genre_mapping[movie_id].append(genre_id)

data = []
for movie_id, genre_list in movie_genre_mapping.items():
    for genre_id in genre_list:
        data.append({'movie_id': movie_id, 'genre_id': genre_id})

movie_genres_df = pd.DataFrame(data)

movie_genres_df.to_csv('movie_genres.csv', index=False) 
print(movie_genres_df)

       movie_id  genre_id
0             1         1
1             1         2
2             1         3
3             1         4
4             2         1
...         ...       ...
14499      5041         5
14500      5042         9
14501      5042        14
14502      5042         7
14503      5043         6

[14504 rows x 2 columns]


# ratings

In [56]:
df['imdb_score'] = df['imdb_score'].replace(9.9, 10)
ratings = pd.DataFrame()
ratings['movie_id'] = df['index'] + 1
ratings['movie_title'] = df['movie_title'] 
ratings['rating'] = df['imdb_score']

ratings.drop_duplicates(subset=['movie_title'])
print(ratings)

ratings.to_csv('ratings.csv', index=False)

      movie_id                                        movie_title  rating
0            1                                            Avatar      7.9
1            2          Pirates of the Caribbean: At World's End      7.1
2            3                                           Spectre      6.8
3            4                             The Dark Knight Rises      8.5
4            5  Star Wars: Episode VII - The Force Awakens    ...     7.1
...        ...                                                ...     ...
5038      5039                           Signed Sealed Delivered      7.7
5039      5040                         The Following                  7.5
5040      5041                              A Plague So Pleasant      6.3
5041      5042                                  Shanghai Calling      6.3
5042      5043                                 My Date with Drew      6.6

[5043 rows x 3 columns]
